Approach 1: treat all rows independently
Approach 2: keep posts and comments separate
Approach 3: combine posts and comments

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import nltk
from nltk.corpus import stopwords
import spacy
from gensim.models import CoherenceModel
import pyLDAvis.gensim
import pyLDAvis
import warnings
import matplotlib.pyplot as plt

# nltk.download('stopwords')

In [32]:
file_path = 'C:/Users/user/OneDrive - Oulun yliopisto/Documents/suomi24/Data/suomi24.csv'
df = pd.read_csv(file_path)
df

,msg_type,datetime,title,thread_id,comment_id,topic_name_top,topic_name_leaf,thread_text
0,thread_start,2001-01-18 17:18:00,Questions about Estonia,89742,0,Society,# The world's going on #,- Why did an important piece of evidence (from...
1,comment,2001-01-19 01:28:00,Questions about Estonia,89742,386913,Society,# The world's going on #,I think they had other things to do. - First o...
2,comment,2001-01-19 06:21:00,Questions about Estonia,89742,386914,Society,# The world's going on #,"- Not the bow gate, but its SARANA- &gt; a str..."
3,comment,2001-01-19 16:27:00,Questions about Estonia,89742,386915,Society,# The world's going on #,I'll tell you what's going on. I'll tell you w...
4,comment,2001-01-20 12:49:00,Questions about Estonia,89742,386916,Society,# The world's going on #,Let the cause of the sinking of any ship be li...
...,...,...,...,...,...,...,...,...
274514,comment,2020-12-31 17:08:44,Thai rice dogs,16711386,105838676,Tourism,Pattaya,"as if these omega- ""men"" were taken there. The..."
274515,thread_start,2020-12-30 17:58:06,Surveillance camera,16711801,0,Home and construction,Lighting and electrical works,Surveillance camera Is there a human next outs...
274516,comment,2020-12-31 06:34:39,Surveillance camera,16711801,105829873,Home and construction,Lighting and electrical works,Pretty sure the camera will follow you. Put th...
274517,comment,2020-12-31 13:27:04,Surveillance camera,16711801,105835154,Home and construction,Lighting and electrical works,please see online if you have Internet access ...


Preprocess

In [33]:
# Basic text preprocessing function
stop_words = set(stopwords.words('english'))
stop_words.add('s')
stop_words.add('ve')
stop_words.add('re')
stop_words.add('m')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lower case
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\b\w{1,2}\b', '', text) # Remove single and double character words
    text = text.lower()  # Convert to lower case
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the 'thread_text' column
df['processed_text'] = df['thread_text'].astype(str).apply(preprocess_text)


In [34]:
df['processed_text'][0]

' why did an important piece of evidence from the front gate  fall down into the depths of the sea from the helicopter that carried it questions about estonia  why isnt estonia the only ship in the world that immediately held up the peace of the grave  why did the border guard prevent investigations by an independent tv company before the area was protected  why is the socalled burial peace repeatedly invoked even though most of the relatives wanted to study the matter  why only one theory of sinking even though it is very controversial  flashed into the public domain as truth  didnt anyone get saved from the bridge of command even though they were in the advantage  why were no other images given to the public when they had a couple of stormy images'

In [35]:
# Define a function to filter relevant rows
def is_relevant(text):
    # Example criterion: contains certain keywords
    keywords = ['camera', 'public', 'privacy', 'cctv', 'surveillance', 'security', 'safety', 'concern', 'protect', 'monitoring', 'footage', 'personal', 'cameras', 'concerns', 'protection', 'monitor']
    return any(keyword in text for keyword in keywords)

# Apply the function to filter rows
df_relevant = df[df['processed_text'].apply(is_relevant)]

In [36]:
df_relevant

,msg_type,datetime,title,thread_id,comment_id,topic_name_top,topic_name_leaf,thread_text,processed_text
0,thread_start,2001-01-18 17:18:00,Questions about Estonia,89742,0,Society,# The world's going on #,- Why did an important piece of evidence (from...,why did an important piece of evidence from t...
15,comment,2001-01-22 11:16:00,Questions about Estonia,89742,386927,Society,# The world's going on #,intutorytutkimus. fi and there MV Estonia. The...,intutorytutkimus fi and there mv estonia the f...
27,thread_start,2001-01-19 12:15:00,automatic control is just money laundering,32552,0,Vehicles and transport,General information on driving,car thieves and so on. But when speed restrict...,car thieves and so on but when speed restricti...
28,comment,2001-01-19 12:59:00,automatic control is just money laundering,32552,162166,Vehicles and transport,General information on driving,how can you teach your children to walk throug...,how can you teach your children to walk throug...
31,comment,2001-01-19 17:05:00,automatic control is just money laundering,32552,162170,Vehicles and transport,General information on driving,even though I don't have a radar detector or e...,even though i dont have a radar detector or ev...
...,...,...,...,...,...,...,...,...,...
274498,comment,2020-12-31 10:33:44,Thai rice dogs,16711386,105832145,Tourism,Pattaya,"quiet area, but now I have a house at three me...",quiet area but now i have a house at three met...
274502,comment,2020-12-31 13:32:47,Thai rice dogs,16711386,105835231,Tourism,Pattaya,At that time they were expensive devices. Well...,at that time they were expensive devices well ...
274507,comment,2020-12-31 14:34:39,Thai rice dogs,16711386,105836278,Tourism,Pattaya,"someone doesn't match. . Earlier, you told me ...",someone doesnt match earlier you told me you ...
274515,thread_start,2020-12-30 17:58:06,Surveillance camera,16711801,0,Home and construction,Lighting and electrical works,Surveillance camera Is there a human next outs...,surveillance camera is there a human next outs...


Topic Modeling

In [37]:
def sent_to_words(texts):
    for text in texts:
        yield(simple_preprocess(str(text),deacc=True))  # deacc=True removes the punctuation marks
        
data_words=list(sent_to_words(df['processed_text']))
# data_words=list(sent_to_words(df_relevant['processed_text']))

In [38]:
data_words

[['why',
  'did',
  'an',
  'important',
  'piece',
  'of',
  'evidence',
  'from',
  'the',
  'front',
  'gate',
  'fall',
  'down',
  'into',
  'the',
  'depths',
  'of',
  'the',
  'sea',
  'from',
  'the',
  'helicopter',
  'that',
  'carried',
  'it',
  'questions',
  'about',
  'estonia',
  'why',
  'isnt',
  'estonia',
  'the',
  'only',
  'ship',
  'in',
  'the',
  'world',
  'that',
  'immediately',
  'held',
  'up',
  'the',
  'peace',
  'of',
  'the',
  'grave',
  'why',
  'did',
  'the',
  'border',
  'guard',
  'prevent',
  'investigations',
  'by',
  'an',
  'independent',
  'tv',
  'company',
  'before',
  'the',
  'area',
  'was',
  'protected',
  'why',
  'is',
  'the',
  'socalled',
  'burial',
  'peace',
  'repeatedly',
  'invoked',
  'even',
  'though',
  'most',
  'of',
  'the',
  'relatives',
  'wanted',
  'to',
  'study',
  'the',
  'matter',
  'why',
  'only',
  'one',
  'theory',
  'of',
  'sinking',
  'even',
  'though',
  'it',
  'is',
  'very',
  'controvers

In [39]:
stop_words=stopwords.words('english')
stop_words.append('s')
stop_words.append('ve')
stop_words.append('re')
stop_words.append('m')
print(stop_words)
# remove stopwords
def remove_stopwords(texts):
    return [[word for word in text if word not in stop_words] for text in texts]

data_words_nostops=remove_stopwords(data_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
len(data_words_nostops)

274519

In [ ]:
data_words_nostops

In [42]:
# Applying Bigrams and trigrams 
bigram=gensim.models.Phrases(data_words_nostops,min_count=5,threshold=80)
trigram=gensim.models.Phrases(bigram[data_words_nostops],threshold=80)

bigram_mod=gensim.models.phrases.Phraser(bigram)

def make_bigram(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_nostops_bigrams=make_bigram(data_words_nostops)

In [43]:
bigram_phrases = [phrase for phrase, score in bigram.vocab.items() if '_' in phrase]

In [ ]:
bigram_phrases

In [45]:
len(data_words_nostops_bigrams)

274519

In [ ]:
data_words_nostops_bigrams

In [47]:
nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])

def lemmatize(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out=[]
    for sent in texts:
        doc=nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return texts_out

In [48]:
data_lemmatized=lemmatize(data_words_nostops_bigrams,allowed_postags=['NOUN','ADJ','VERB','ADV'] )

In [ ]:
data_lemmatized

In [50]:
id2word=corpora.Dictionary(data_lemmatized)
texts=data_lemmatized  # list of list of tokens
corpus=[id2word.doc2bow(text) for text in texts]
print(corpus[:1])   #first document

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)]]


In [51]:
id2word[0]

'advantage'

In [52]:
print([[(id2word[id],freq) for (id,freq) in cp]for cp in corpus[:2]])

[[('advantage', 1), ('area', 1), ('border_guard', 1), ('bridge', 1), ('burial_peace', 1), ('carry', 1), ('command', 1), ('company', 1), ('controversial', 1), ('couple', 1), ('depth', 1), ('even', 3), ('evidence', 1), ('fall', 1), ('flash', 1), ('get', 1), ('give', 1), ('grave', 1), ('helicopter', 1), ('hold', 1), ('image', 2), ('immediately', 1), ('important', 1), ('independent', 1), ('investigation', 1), ('invoke', 1), ('matter', 1), ('peace', 1), ('piece', 1), ('prevent', 1), ('protect', 1), ('public', 1), ('public_domain', 1), ('question', 1), ('relative', 1), ('repeatedly', 1), ('save', 1), ('sea', 1), ('ship', 1), ('sink', 1), ('socalle', 1), ('stormy', 1), ('study', 1), ('theory', 1), ('truth', 1), ('want', 1), ('world', 1)], [('company', 1), ('independent', 1), ('save', 1), ('stormy', 3), ('theory', 1), ('able', 1), ('accident', 1), ('addition', 1), ('amazing', 1), ('argument', 1), ('authority', 1), ('bottom', 1), ('continue', 1), ('demand', 1), ('describe', 1), ('dive', 1), ('e

In [58]:
# building LDA Model
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=3,random_state=100,update_every=0,
                                         chunksize=500,passes=20,alpha='auto',per_word_topics=True)


In [59]:
print(lda_model.print_topics())

[(0, '0.265*"money" + 0.038*"lot" + 0.036*"bit_bit" + 0.010*"foot_foot" + 0.009*"ill" + 0.007*"spend" + 0.007*"go" + 0.006*"bit" + 0.005*"s" + 0.004*"youll_youll"'), (1, '0.013*"go" + 0.011*"get" + 0.009*"s" + 0.008*"even" + 0.007*"camera" + 0.007*"time" + 0.007*"think" + 0.006*"m" + 0.006*"good" + 0.006*"people"'), (2, '0.014*"order" + 0.010*"accident_accident" + 0.006*"funds_fund" + 0.005*"measure_measure" + 0.002*"go" + 0.002*"amount_amount" + 0.002*"es_es" + 0.002*"european_central" + 0.002*"sleet_sleet" + 0.001*"m"')]


In [60]:
doc_lda=lda_model[corpus]
print(list(doc_lda)[0])

([(1, 0.9998964)], [(0, [1]), (1, [1]), (2, [1]), (3, [1]), (4, [1]), (5, [1]), (6, [1]), (7, [1]), (8, [1]), (9, [1]), (10, [1]), (11, [1]), (12, [1]), (13, [1]), (14, [1]), (15, [1]), (16, [1]), (17, [1]), (18, [1]), (19, [1]), (20, [1]), (21, [1]), (22, [1]), (23, [1]), (24, [1]), (25, [1]), (26, [1]), (27, [1]), (28, [1]), (29, [1]), (30, [1]), (31, [1]), (32, [1]), (33, [1]), (34, [1]), (35, [1]), (36, [1]), (37, [1]), (38, [1]), (39, [1]), (40, [1]), (41, [1]), (42, [1]), (43, [1]), (44, [1]), (45, [1]), (46, [1])], [(0, [(1, 0.9992476)]), (1, [(1, 0.99989897)]), (2, [(1, 0.9926466)]), (3, [(1, 0.99884176)]), (4, [(1, 0.9419987)]), (5, [(1, 0.999768)]), (6, [(1, 0.99864477)]), (7, [(1, 0.9998896)]), (8, [(1, 0.99118984)]), (9, [(1, 0.99986506)]), (10, [(1, 0.99729055)]), (11, [(1, 2.9999566)]), (12, [(1, 0.99974227)]), (13, [(1, 0.99983317)]), (14, [(1, 0.99965)]), (15, [(1, 0.9999895)]), (16, [(1, 0.9999449)]), (17, [(1, 0.99797076)]), (18, [(1, 0.9978397)]), (19, [(1, 0.9996806

In [61]:
#LDA Model Perfromance check
print("Perplexity: ",lda_model.log_perplexity(corpus))

#compute coherence score
coherence_model_lda=CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')

coherence_lda=coherence_model_lda.get_coherence()
print('Coherence Score: ',coherence_lda)

Perplexity:  -7.836957345077162
Coherence Score:  0.5238194223644014


In [62]:
warnings.filterwarnings('ignore',category=FutureWarning)

pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.266121 -0.013587       1        1  98.752939
0     -0.163809 -0.081401       2        1   1.107318
2     -0.102312  0.094988       3        1   0.139743, topic_info=       Term          Freq         Total Category  logprob  loglift
478   money  35582.000000  35582.000000  Default  30.0000  30.0000
116      go  69528.000000  69528.000000  Default  29.0000  29.0000
127     lot  18957.000000  18957.000000  Default  28.0000  28.0000
119       s  49581.000000  49581.000000  Default  27.0000  27.0000
876   order   6455.000000   6455.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
119       s      6.378865  49581.474975   Topic3  -7.1100  -2.3853
93     good      6.045851  33552.289907   Topic3  -7.1636  -2.0484
189  people      5.976546  32870.347562   Topic3  -7.1751  -2.0394
157    time      5.843263  37650.708938   Topic3  -7.1977  -2.1977
79     take      5.269857  29803.633888   Topic3  -7.3009  -2.0673

[241 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
7582      1  0.829863  accident_accident
7582      3  0.169087  accident_accident
162       1  0.998888               also
162       2  0.000992               also
162       3  0.000110               also
...     ...       ...                ...
8700      1  0.548396               youm
8700      2  0.456997               youm
678       1  0.913261               yous
678       2  0.085466               yous
678       3  0.002442               yous

[348 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])